# Comparing the Quality and Loess between a Full and a Half Tesla Cavity Using the EPR approach
Calculation of resonant cavity quality factors, losses, life-time and more using the EPR method. This code is based on [Ansys HFSS](https://www.ansys.com/products/electronics/ansys-hfss) and the [pyEPR library](https://github.com/zlatko-minev/pyEPR).

The cavity is half of a [Tesla cavity](https://arxiv.org/pdf/physics/0003011.pdf), this will be usefule in a later notebook where we compare the full Tesla cavity to our (half Tesla) cavity.

#### Imports

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy = True

In [2]:
import sys
import numpy as np
from IPython.display import display, Math, Latex, display_markdown
from pathlib import Path
import pandas as pd
from scipy import constants

import pyEPR as epr
from pyEPR.calcs import Convert
from pyEPR.core import *
from pyEPR.ansys import *
import warnings
warnings.simplefilter("ignore")

In [3]:
# path_to_project = 'D:\\Users\\Daniel\\Cavity-Analysis'
path_to_project = '.'

## 👉 Half Cavity
We start with the interesting one, the half cavity. This is the cavity we're going to be using and we want to check that it isn't that much worse than the full Tesla cavity.

### 🔷 Mode analysis

#### 🔹 Connect to HFSS

In [4]:
pinfo = epr.Project_Info(project_path = path_to_project, 
                         project_name = 'Cavity Analysis',
                         design_name  = 'half cav')

INFO 11:21AM [connect]: Connecting to Ansys Desktop API...
INFO 11:21AM [load_ansys_project]: 	File path to HFSS project found.
INFO 11:21AM [load_ansys_project]: 	Opened Ansys App
INFO 11:21AM [load_ansys_project]: 	Opened Ansys Desktop v2020.1.0
INFO 11:21AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/Daniel/Cavity-Analysis/
	Project:   Cavity Analysis
INFO 11:21AM [connect]: 	Opened active design
	Design:    half cav [Solution type: Eigenmode]
INFO 11:21AM [get_setup]: 	Opened setup `setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:21AM [connect]: 	Connection to Ansys established successfully. 😀 



In [5]:
pinfo.setup.analyze()
eprh = epr.DistributedAnalysis(pinfo)

INFO 11:21AM [analyze]: Analyzing setup setup


Design "half cav" info:
	# eigenmodes    1
	# variations    2


#### 🔹 Get HFSS mode and quality results

In [6]:
modes      = eprh.get_freqs_bare_pd(eprh.variations[0])
Fs, Qs     = np.array(modes['Freq. (GHz)']), np.array(modes['Quality Factor'])  # Get freqs and Q-factors
n_modes    = int(pinfo.setup.n_modes)
display(modes)

,Freq. (GHz),Quality Factor
mode,,
0,4.069412,1248.378885


#### 🔹 Calculate the EPRs of the modes

In [7]:
eprh.set_mode(0)
p_cavity, (ℰ_cav, ℰ_total) = eprh.calc_p_electric_volume('cavity')
p_dirt,  (ℰ_dirt, ℰ_total) = eprh.calc_p_electric_volume('dirt')

print(f' 🔸 Energy in cavity = {100*p_cavity:.3f}% -> {ℰ_cav:0.2e} of the total energy in the system')
print(f' 🔸 Energy in dirt   = {100*p_dirt:.3f}% -> {ℰ_dirt:0.2e} of the total energy in the system')
print(f' 🔸 Total energy     = {ℰ_total:0.2e}')

 🔸 Energy in cavity = 95.995% -> 8.11e-17 of the total energy in the system
 🔸 Energy in dirt   = 4.005% -> 3.38e-18 of the total energy in the system
 🔸 Total energy     = 8.45e-17


### 🔷 Life-times

**Life-time from HFSS**

Life time of a mode inside the cavity. Since in this exampole the cavity is *perfect*, the life time would be infinite

In [8]:
Fs_Hz  = np.array(Convert.toSI(Fs,'GHz'))  # Mode freqs in Hz
omegas = 2*np.pi*Fs_Hz                     # Freqs to angular freqs
taus   = Qs/omegas                         # Life times

print(f' 🔸 Life-time of mode = {taus[0]*1e3:.3f} ms')  # Should be inf since no resistive boundry and just inside a vacuum

 🔸 Life-time of mode = 0.000 ms


**Life-time from EPR**

Life time calculation with the EPR method. This is highly dependent on the loss tangent of the dirt and cavity.

In [9]:
tan_dirt   = 4e-7                                         # Loss tangent of dirt

tau_epr    = lambda p, tan, omega: 1/(p*tan*omega)        # Easily calculate life time with EPR

tau_cavity = tau_epr(p_dirt, 0, omegas)[0]
tau_dirt   = tau_epr(p_cavity, tan_dirt, omegas)[0]

print(f' 🔸 Cavity life-time = {tau_cavity*1e6:.2f} ns')  # Should be infinite since the cavity is a pefect vacum
print(f' 🔸 Dirt life-time   = {tau_dirt*1e6:.2f} ns')

 🔸 Cavity life-time = inf ns
 🔸 Dirt life-time   = 101.85 ns


### 🔷 Losses

#### 🔹 Surface loss

Calculating the energy precentage near the cavity walls by the surface integral. The total energy of the electromagnetic field at a layer of thickness `dirt_widht` would be approximated as:
$$\text{E}_{\text{cavity, boundry}} \approx \text{dirt_width} \cdot \int_{S_{cavity}} |E|^2$$
$$\text{E}_{\text{cavity, volume}} = \int_{V_{cavity}} |E|^2$$

$$\text{EPR}_{\text{cavity, boundry}} \approx \frac{\text{E}_{\text{cavity, boundry}}}{\text{E}_{\text{cavity, volume}}}$$

First we'll setup a dictionary to store all the data

In [10]:
data = {
    "half":{
        "volume":{
            
        },
        "surface":{
           
        }
    },
    "full":{
        "volume":{
          
        },
        "surface":{
           
        }
    }
}

In [11]:
dirt_width = 0.1e-3
eps        = 1
tan_surf   = 5e-3

eprh.set_mode(0)

# --- Surface integral ---
surf = 'cavity'
calcobject = CalcObject([], eprh.setup)
vecE = calcobject.getQty("E").smooth()
A = vecE.times_eps()
B = vecE.conj()
A = A.dot(B)
A = A.real()
A = A.integrate_surf(name=surf)

E_subs = A.evaluate(lv=eprh._get_lv()) 
E_surf = E_subs*dirt_width*eps

# --- Volume integral ---
E_total = eprh.calc_energy_electric(smooth=True)

p_surf = E_surf/E_total      # EPR of surface 
Q_surf = 1/tan_surf/p_surf   # Q-fact of surface
tau_surf = Q_surf/omegas[0]  #  Life-time of surface

data['half']['surface'] = {
    "EPR": p_surf,
    "Q":   Q_surf,
    "tau":  tau_surf
}

print(f' 🔸 EPR surface       = {100*p_surf:.2f}%')
print(f' 🔸 Q-factor surface  = {Q_surf:.2e}')
print(f' 🔸 Life-time surface = {tau_surf:.2e} seconds \n')


 🔸 EPR surface       = 1.54%
 🔸 Q-factor surface  = 1.30e+04
 🔸 Life-time surface = 5.07e-07 seconds 



#### 🔹 Dirt (volume) loss

In [12]:
# Dirt is simulated as much thicker than it actually is (for computation reason). 
# Beacuase of that we reduce the loss tangent to an 'effective loss tangent' which is loss_tan*thick_factor
p_dirt, (ℰ_dirt, ℰ_total) = eprh.calc_p_electric_volume('dirt')
Q_dirt = 1/(tan_surf*p_dirt)
tau_dirt = Q_dirt/omegas[0]

data['half']['volume'] = {
    "EPR": p_dirt,
    "Q":   Q_dirt,
    "tau":  tau_dirt
}

print(f'  🔸 EPR of dirt    = {100*p_dirt:0.2f}% ( {ℰ_dirt:.2e} / {ℰ_total:.2e} )')
print(f'  🔸 Quality factor = {Q_dirt:0.2e}')
print(f'  🔸 life time      = {tau_dirt:0.2e} seconds\n')

  🔸 EPR of dirt    = 4.01% ( 3.38e-18 / 8.45e-17 )
  🔸 Quality factor = 4.99e+03
  🔸 life time      = 1.95e-07 seconds



## 👉 Full Cavity

### 🔷 Mode analysis

#### 🔹 Connect to HFSS

In [13]:
pinfo_full = epr.Project_Info(project_path = path_to_project, 
                             project_name = 'Cavity Analysis',
                             design_name  = 'full cav')

INFO 11:21AM [connect]: Connecting to Ansys Desktop API...
INFO 11:21AM [load_ansys_project]: 	File path to HFSS project found.
INFO 11:21AM [load_ansys_project]: 	Opened Ansys App
INFO 11:21AM [load_ansys_project]: 	Opened Ansys Desktop v2020.1.0
INFO 11:21AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/Daniel/Cavity-Analysis/
	Project:   Cavity Analysis
INFO 11:21AM [connect]: 	Opened active design
	Design:    full cav [Solution type: Eigenmode]
INFO 11:21AM [get_setup]: 	Opened setup `setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:21AM [connect]: 	Connection to Ansys established successfully. 😀 



In [14]:
pinfo_full.setup.analyze()
eprh_full = epr.DistributedAnalysis(pinfo_full)

INFO 11:21AM [analyze]: Analyzing setup setup


Design "full cav" info:
	# eigenmodes    1
	# variations    2


#### 🔹 Get HFSS mode and quality results

In [15]:
modes      = eprh_full.get_freqs_bare_pd(eprh_full.variations[0])
Fs, Qs     = np.array(modes['Freq. (GHz)']), np.array(modes['Quality Factor'])  # Get freqs and Q-factors
n_modes    = int(pinfo_full.setup.n_modes)
display(modes)

,Freq. (GHz),Quality Factor
mode,,
0,4.236008,2552.281993


#### 🔹 Calculate the EPRs of the modes

In [16]:
eprh_full.set_mode(0)
p_cavity, (ℰ_cav, ℰ_total) = eprh_full.calc_p_electric_volume('cavity')
p_dirt, (ℰ_dirt, ℰ_total) = eprh_full.calc_p_electric_volume('dirt')

print(f' 🔸 Energy in cavity = {100*p_cavity:.3f}% -> {ℰ_cav:0.2e}')
print(f' 🔸 Energy in dirt   = {100*p_dirt:.3f}% -> {ℰ_dirt:0.2e}')
print(f' 🔸 Total energy     = {ℰ_total:0.2e}')

 🔸 Energy in cavity = 98.041% -> 1.66e-16
 🔸 Energy in dirt   = 1.959% -> 3.31e-18
 🔸 Total energy     = 1.69e-16


### 🔷 Life-times

**Life-time from HFSS**

In [17]:
Fs_Hz  = np.array(Convert.toSI(Fs,'GHz'))  # Mode freqs in Hz
omegas = 2*np.pi*Fs_Hz                     # Freqs to angular freqs
taus   = Qs/omegas                         # Life times

print(f' 🔸 Life-time of mode = {taus[0]*1e3:.3f} ms')

 🔸 Life-time of mode = 0.000 ms


**Life-time from EPR**

In [18]:
tan_dirt = 4e-7  # Loss tangent of dirt

tau_epr = lambda p, tan, omega: 1/(p*tan*omega)  # Easily calculate life time with EPR

tau_cavity = tau_epr(p_dirt, 0, omegas)[0]
tau_dirt = tau_epr(p_cavity, tan_dirt, omegas)[0]

print(f' 🔸 Cavity life-time = {tau_cavity*1e6:.2f} ns')  # Should be infinite since the cavity is a pefect vacum
print(f' 🔸 Dirt life-time   = {tau_dirt*1e6:.2f} ns')

 🔸 Cavity life-time = inf ns
 🔸 Dirt life-time   = 95.81 ns


### 🔷 Losses

#### 🔹 Surface loss

In [19]:
eprh_full.set_mode(0)

# --- Surface integral ---
surf = 'cavity'
calcobject = CalcObject([], eprh_full.setup)
vecE = calcobject.getQty("E").smooth()
A = vecE.times_eps()
B = vecE.conj()
A = A.dot(B)
A = A.real()
A = A.integrate_surf(name=surf)

E_subs = A.evaluate(lv=eprh_full._get_lv()) 
E_surf = E_subs*dirt_width*eps

# --- Volume integral ---
E_total = eprh_full.calc_energy_electric(smooth=True)

p_surf = E_surf/E_total      # EPR of surface 
Q_surf = 1/tan_surf/p_surf   # Q-fact of surface
tau_surf = Q_surf/omegas[0]  #  Life-time of surface

data['full']['surface'] = {
    "EPR":  p_surf,
    "Q":    Q_surf,
    "tau":  tau_surf
}

print(f' 🔸 EPR surface       = {100*p_surf:.2f}%')
print(f' 🔸 Q-factor surface  = {Q_surf:.2e}')
print(f' 🔸 Life-time surface = {tau_surf:.2e} seconds \n')

 🔸 EPR surface       = 0.70%
 🔸 Q-factor surface  = 2.86e+04
 🔸 Life-time surface = 1.07e-06 seconds 



#### 🔹 Dirt (volume) loss

In [20]:
# Dirt is simulated as much thicker than it actually is (for computation reason). 
# Beacuase of that we reduce the loss tangent to an 'effective loss tangent' which is loss_tan*thick_factor
p_dirt, (ℰ_dirt, ℰ_total) = eprh_full.calc_p_electric_volume('dirt')
Q_dirt = 1/(tan_surf*p_dirt)
tau_dirt = Q_dirt/omegas[0]

data['full']['volume'] = {
    "EPR":  p_dirt,
    "Q":    Q_dirt,
    "tau":  tau_dirt
}

print(f'  🔸 EPR of dirt    = {100*p_dirt:0.2f}% ( {ℰ_dirt:.2e} / {ℰ_total:.2e} )')
print(f'  🔸 Quality factor = {Q_dirt:0.2e}')
print(f'  🔸 life time      = {tau_dirt:0.2e} seconds\n')

  🔸 EPR of dirt    = 1.96% ( 3.31e-18 / 1.69e-16 )
  🔸 Quality factor = 1.02e+04
  🔸 life time      = 3.84e-07 seconds



## Conclusion 🎉
We'll compare the full cavity to the half cavity now. First with the surface integral calculation, then with the volume calculation and finally we'll compare the two approaches (that should yield roughly the same result). We'll calculate the ratio between the full cavity calculated parameters and half cavity calculated parameters. 

In [21]:
# ═══ Surface integral ═══
EPR_ratio_surf  = data['full']['surface']['EPR']/data['half']['surface']['EPR']
Q_ratio_surf    = data['full']['surface']['Q']/data['half']['surface']['Q']
tau_ratio_surf  = data['full']['surface']['tau']/data['half']['surface']['tau']

# ═══ Volume integral ═══
EPR_ratio_vol   = data['full']['volume']['EPR']/data['half']['volume']['EPR']
Q_ratio_vol     = data['full']['volume']['Q']/data['half']['volume']['Q']
tau_ratio_vol   = data['full']['volume']['tau']/data['half']['volume']['tau']

# ═══ Volume vs Surface ═══
EPR_surf_vol    = EPR_ratio_surf/EPR_ratio_vol
Q_surf_vol      = Q_ratio_surf/Q_ratio_vol
tau_surf_vol    = tau_ratio_surf/tau_ratio_vol

print('═'*10, '🎉 Results 🎉', '═'*10)
print('Ratio full-to-half cavity parameters\n')
print('\t─── Surface integral ───')
print(f'  🔸 EPR:       {EPR_ratio_surf:.3f}')
print(f'  🔸 Q-factor:  {Q_ratio_surf:.3f}')
print(f'  🔸 Life-time: {tau_ratio_surf:.3f}\n')

print('\t─── Volume integral ───')
print(f'  🔸 EPR:       {EPR_ratio_vol:.3f}')
print(f'  🔸 Q-factor:  {Q_ratio_vol:.3f}')
print(f'  🔸 Life-time: {tau_ratio_vol:.3f}\n')

print('\t─── Volume v Surface ───')
print(f'  🔸 EPR:       {EPR_surf_vol:.3f}')
print(f'  🔸 Q-factor:  {Q_surf_vol:.3f}')
print(f'  🔸 Life-time: {tau_surf_vol:.3f}')

══════════ 🎉 Results 🎉 ══════════
Ratio full-to-half cavity parameters

	─── Surface integral ───
  🔸 EPR:       0.454
  🔸 Q-factor:  2.204
  🔸 Life-time: 2.117

	─── Volume integral ───
  🔸 EPR:       0.489
  🔸 Q-factor:  2.044
  🔸 Life-time: 1.964

	─── Volume v Surface ───
  🔸 EPR:       0.928
  🔸 Q-factor:  1.078
  🔸 Life-time: 1.078


In [28]:
print('Ratio surface to volume of EPR')
print(f'  🔸 Half cavity:  {data["half"]["surface"]["EPR"]/data["half"]["volume"]["EPR"]:.3f}')  # <-- Good
print(f'  🔸 Full cavity:  {data["full"]["surface"]["EPR"]/data["full"]["volume"]["EPR"]:.3f}')  # <-- Bad

Ratio surface to volume of EPR
  🔸 Half cavity:  0.385
  🔸 Full cavity:  0.357


In [47]:
print('\t─── Full ───')
print(f'EPR        = {np.average([data["full"]["surface"]["EPR"], data["full"]["volume"]["EPR"]])*100:.2f}%')
print(f'Q-factor   = {np.average([data["full"]["surface"]["Q"], data["full"]["volume"]["Q"]]):.2e}')
print(f'Life-time  = {np.average([data["full"]["surface"]["tau"], data["full"]["volume"]["tau"]])*1e9:.2e}ns')

print('\n\t─── Half ───')
print(f'EPR        = {np.average([data["half"]["surface"]["EPR"], data["half"]["volume"]["EPR"]])*100:.2f}%')
print(f'Q-factor   = {np.average([data["half"]["surface"]["Q"], data["half"]["volume"]["Q"]]):.2e}')
print(f'Life-time  = {np.average([data["half"]["surface"]["tau"], data["half"]["volume"]["tau"]])*1e9:.2e} ns')

	─── Full ───
EPR        = 1.33%
Q-factor   = 1.94e+04
Life-time  = 7.28e+02ns

	─── Half ───
EPR        = 2.77%
Q-factor   = 8.98e+03
Life-time  = 3.51e+02 ns


In [35]:
data["full"]["volume"]["EPR"]

0.019590281694566412

In [23]:
import pprint
print('\t \33[4mRaw data\33[0m')
pprint.pprint(data)

	 Raw data
{'full': {'surface': {'EPR': 0.007001279686534065,
                      'Q': 28566.20631577834,
                      'tau': 1.0732870954920978e-06},
          'volume': {'EPR': 0.019590281694566412,
                     'Q': 10209.143651848164,
                     'tau': 3.835770846302925e-07}},
 'half': {'surface': {'EPR': 0.015428110159270167,
                      'Q': 12963.35052934708,
                      'tau': 5.069973700805057e-07},
          'volume': {'EPR': 0.04005192421841798,
                     'Q': 4993.51788716382,
                     'tau': 1.9529676610307113e-07}}}


In [24]:
pinfo.disconnect()
pinfo_full.disconnect()

## Notes on the design 📝
To simplify the problem as much as possible, the cavity is just a 'squashed' circle.

**Create cavity:** The steps to create the cavity are as follows:
  * Create an elipse with it's major axis being the radius of the cavity, and the minor axis the 'squashed' radius.
  * Create a rectengle in the plane of the cavity so that the intersection between the elipse and the rectangle is half of the elipse (symetric along the minor axis)
  * Select the rectangel and elipse and choose `Draw`->`Intersect`, creating a new object that is the intersect between them.
  * Choose the newly created object and click on `Draw`->`sweep around axis` (the folding sheet icon) to create a 3D 'squashed' circle.

To create the half cavity, all you need is to move around the intersecting rectange so you're left with a quarter of the original elipse.


**Create dirt:** To create the dirt with constant width, the easiest method is by following these steps:
  * Duplicate the cavity object once
  * Right click on the duplicate in the side-bar and choose `Select`->`All Faces`.
  * Once all the faces are selected you can press `Draw`->`Surface`->`Move Faces Along Normal`, a window will pop up promting you the write a distance, this would be the thickness of the dirt `dirt_width` (make sure it's a negative number).
  * Take the newly created smaller cavity and duplicate it, this would be the vacuum. Now select the original cavity (the big one) and one of the smaller ones together and press `Draw`->`Subtract` (make sure to do so in the correct order), this would make a new object which will be our dirt.
 
 You should be left with one object being the cavity and one object being the dirt.
 
 $\dagger$ Make sure that the cavity and the dirt don't intersect, if they do, you might get wildly wrong solutions
  